数据集需要通过标记化，才能给模型拿去训练。<br/>
这里看几个数据集的处理方式。<br/>
#### 对整个数据集进行标记化

In [1]:
from datasets import load_dataset
emotions = load_dataset("emotion")

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
No config specified, defaulting to: emotion/split
Found cached dataset emotion (/home/trs/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from transformers import AutoTokenizer 
model_ckpt = "distilbert-base-uncased" 
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

为了对整个语料库进行标记，我们将使用DatasetDict对象的map（）方法。<br/>
map()方法也可以用来创建新的行和列。

这个函数将标记器应用于一批例子。 padding=True将用零填充例子，使其达到一个批次中最长的一个的大小，truncation=True将把例子截断到模型的最大上下文大小。

In [3]:
def tokenize(batch): 
	return tokenizer(batch["text"], padding=True, truncation=True)

在下面这个调用中，将emotions数据集中train部分的text文本传了进去。

In [4]:
print(tokenize(emotions["train"][:2]))

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


这里我们可以看到填充的结果。 input_ids的第一个元素比第二个短，所以在这个元素上加了零，使它们的长度相同。 这些零在词汇中都有一个相应的[PAD]标记，特殊标记集还包括我们之前遇到的[CLS]和[SEP]标记。<br/>
还要注意的是，除了返回编码后的推文作为input_ids外，tokenizer还返回一个attention_mask数组的列表。 这是因为我们不希望模型被额外的填充标记所迷惑。 attention_mask中填充的部分写的是0。

一旦我们定义了一个处理函数，我们就可以用一行代码将其应用于语料库中的所有分片

In [5]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

默认情况下，map()方法对语料库中的每个例子进行单独操作，所以设置batched=True将对推文进行分批编码。 因为我们设置了batch_size=None，我们的tokenize()函数将作为一个批次应用于整个数据集。 这确保了输入张量和注意力掩码在全球范围内具有相同的形状，我们可以看到这个操作为数据集增加了新的输入_ids和注意力掩码列:

In [6]:
print(emotions_encoded["train"].column_names) 

['text', 'label', 'input_ids', 'attention_mask']


可以看到['input_ids', 'attention_mask']是新增的两列，emotions_encoded是处理后的数据集。<br/>
这样处理后，原只有文本的数据集，就带有标记ID量。

#### 下面我们使用另一个数据集来演示数据集的标记化过程

这里使用的是MRPC数据集中的GLUE 基准测试数据集

In [10]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Found cached dataset glue (/home/trs/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

可以使用它对整个数据集进行处理：如之前所讲，我们可以给标记器提供一组句子，第一个参数是它第一个句子的列表，第二个参数是第二个句子的列表。

In [14]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

这很有效，但它的缺点是返回字典（字典的键是输入词id(input_ids) ， 注意力遮罩(attention_mask) 和 类型标记ID(token_type_ids)，字典的值是键所对应值的列表）（我们需要的是数据集）。<br/>
而且只有当您在转换过程中有足够的内存来存储整个数据集时才不会出错。

#### 动态填充

In [12]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

现在在标记函数中省略了padding参数。这是因为在标记的时候将所有样本填充到最大长度的效率不高。一个更好的做法：在构建批处理时填充样本更好，因为这样我们只需要填充到该批处理中的最大长度，而不是整个数据集的最大长度。<br/>
下面是我们如何在所有数据集上同时应用标记函数。我们在调用map时使用了batch =True，这样函数就可以同时应用到数据集的多个元素上，而不是分别应用到每个元素上。<br/>
这样处理后，返回的就是一个DatasetDict类型数据集，并在数据集里增加了['input_ids', 'token_type_ids', 'attention_mask']字段。

In [15]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/trs/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-b532aa600931f056.arrow
Loading cached processed dataset at /home/trs/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-4bdce1e2012c301e.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

刚才已经说了，在处理数据集时没有进行填充，这里可以看到数组的长度不同。

In [22]:
[len(x) for x in tokenized_datasets["train"]["input_ids"][:10]]

[50, 59, 47, 67, 59, 50, 62, 32, 45, 60]

为了解决句子长度统一的问题，我们必须定义一个collate函数，该函数会将每个batch句子填充到正确的长度。幸运的是，🤗transformer库通过DataCollatorWithPadding为我们提供了这样一个函数。当你实例化它时，需要一个标记器(用来知道使用哪个词来填充，以及模型期望填充在左边还是右边)，并将做你需要的一切:

In [20]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

让我们从我们的训练集中抽取几个样本。这里，我们删除列idx, sentence1和sentence2，因为不需要它们，并查看一个batch中每个条目的长度:

In [23]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

动态填充意味着该批中的所有样本都应该填充到长度为67，这是该批中的最大长度。如果没有动态填充，所有的样本都必须填充到整个数据集中的最大长度，或者模型可以接受的最大长度。让我们再次检查data_collator是否正确地动态填充了这批样本：

In [24]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

In [25]:
batch

{'input_ids': tensor([[  101,  2572,  3217,  5831,  5496,  2010,  2567,  1010,  3183,  2002,
          2170,  1000,  1996,  7409,  1000,  1010,  1997,  9969,  4487, 23809,
          3436,  2010,  3350,  1012,   102,  7727,  2000,  2032,  2004,  2069,
          1000,  1996,  7409,  1000,  1010,  2572,  3217,  5831,  5496,  2010,
          2567,  1997,  9969,  4487, 23809,  3436,  2010,  3350,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [  101,  9805,  3540, 11514,  2050,  3079, 11282,  2243,  1005,  1055,
          2077,  4855,  1996,  4677,  2000,  3647,  4576,  1999,  2687,  2005,
          1002,  1016,  1012,  1019,  4551,  1012,   102,  9805,  3540, 11514,
          2050,  4149, 11282,  2243,  1005,  1055,  1999,  2786,  2005,  1002,
          6353,  2509,  2454,  1998,  2853,  2009,  2000,  3647,  4576,  2005,
          1002,  1015,  1012,  1022,  4551,  1999,  2687, 